In [1]:
# !pip install tensorflow_datasets
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

# Only if you have a GPU

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
######## GPU CONFIGS FOR RTX 2070 ###############
## Please ignore if not training on GPU       ##
## this is important for running CuDNN on GPU ##

# check if GPU can be seen by TF
tf.config.list_physical_devices('GPU')
# enable this line if you wish to see whether GPU/CPU executes
# each command
#tf.debugging.set_log_device_placement(True)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_memory_growth(gpus[0], True)
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)
###############################################

1 Physical GPUs, 1 Logical GPU


# Load Data

In [4]:
# see available tfds data sets
", ".join(tfds.list_builders())

'abstract_reasoning, accentdb, aeslc, aflw2k3d, ag_news_subset, ai2_arc, ai2_arc_with_ir, amazon_us_reviews, anli, arc, bair_robot_pushing_small, bccd, beans, big_patent, bigearthnet, billsum, binarized_mnist, binary_alpha_digits, blimp, bool_q, c4, caltech101, caltech_birds2010, caltech_birds2011, cars196, cassava, cats_vs_dogs, celeb_a, celeb_a_hq, cfq, chexpert, cifar10, cifar100, cifar10_1, cifar10_corrupted, citrus_leaves, cityscapes, civil_comments, clevr, clic, clinc_oos, cmaterdb, cnn_dailymail, coco, coco_captions, coil100, colorectal_histology, colorectal_histology_large, common_voice, coqa, cos_e, cosmos_qa, covid19sum, crema_d, curated_breast_imaging_ddsm, cycle_gan, deep_weeds, definite_pronoun_resolution, dementiabank, diabetic_retinopathy_detection, div2k, dmlab, downsampled_imagenet, dsprites, dtd, duke_ultrasound, emnist, eraser_multi_rc, esnli, eurosat, fashion_mnist, flic, flores, food101, forest_fires, fuss, gap, geirhos_conflict_stimuli, genomics_ood, german_credit

In [5]:
# using TFDS dataset
# note: as_supervised converts dicts to tuples
imdb_train, ds_info = tfds.load(name="imdb_reviews", split="train", 
                                with_info=True, as_supervised=True)
imdb_test = tfds.load(name="imdb_reviews", split="test", as_supervised=True)

In [6]:
print(ds_info)

tfds.core.DatasetInfo(
    name='imdb_reviews',
    version=1.0.0,
    description='Large Movie Review Dataset.
This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.',
    homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
        'text': Text(shape=(), dtype=tf.string),
    }),
    total_num_examples=100000,
    splits={
        'test': 25000,
        'train': 25000,
        'unsupervised': 50000,
    },
    supervised_keys=('text', 'label'),
    citation="""@InProceedings{maas-EtAl:2011:ACL-HLT2011,
      author    = {Maas, Andrew L.  and  Daly, Raymond E.  and  Pham, Peter T.  and  Huang, Dan  and  Ng, Andrew Y.  and  Potts, Christopher},
      title     = {Learning Word

In [7]:
for example, label in imdb_train.take(1):
    print(example, '\n', label)

tf.Tensor(b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.", shape=(), dtype=string) 
 tf.Tensor(0, shape=(), dtype=int64)


In [8]:
# Use the default tokenizer settings
tokenizer = tfds.deprecated.text.Tokenizer()

vocabulary_set = set()
MAX_TOKENS = 0

for example, label in imdb_train:
  some_tokens = tokenizer.tokenize(example.numpy())
  if MAX_TOKENS < len(some_tokens):
        MAX_TOKENS = len(some_tokens)
  vocabulary_set.update(some_tokens)

In [9]:
imdb_encoder = tfds.deprecated.text.TokenTextEncoder(vocabulary_set, 
                                                   tokenizer=tokenizer)
vocab_size = imdb_encoder.vocab_size

print(vocab_size, MAX_TOKENS)

93931 2525


In [10]:
# Lets verify tokenization and encoding works
for example, label in imdb_train.take(1):
    print(example)
    encoded = imdb_encoder.encode(example.numpy())
    print(imdb_encoder.decode(encoded))

tf.Tensor(b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.", shape=(), dtype=string)
This was an absolutely terrible movie Don t be lured in by Christopher Walken or Michael Ironside Both are great actors but this must simply be their worst role in history Even their great acting could not redeem this movie s ridiculous storyline This 

In [11]:
# transformation fucntions to be used with the dataset
from tensorflow.keras.preprocessing import sequence

def encode_pad_transform(sample):
    encoded = imdb_encoder.encode(sample.numpy())
    pad = sequence.pad_sequences([encoded], padding='post', 
                                 maxlen=150)
    return np.array(pad[0], dtype=np.int64)  


def encode_tf_fn(sample, label):
    encoded = tf.py_function(encode_pad_transform, 
                                       inp=[sample], 
                                       Tout=(tf.int64))
    encoded.set_shape([None])
    label.set_shape([])
    return encoded, label

In [12]:
# test the transformation on a small subset
subset = imdb_train.take(10)
tst = subset.map(encode_tf_fn)

In [13]:
for review, label in tst.take(1):
    print(review, label)
    print(imdb_encoder.decode(review))

tf.Tensor(
[13581 47819 62328 56061 52535 30401 66588 43369 89223 35512  5387 16151
 33449 55840 25146 90121 48496 49184 91115 41694 24468 67269   992 86596
   289 89223 81016 32667 16740  5387 11563 63550 81016 41694 19153 39306
 58448 61911   992 30401 10785 84483 84513 13581 30401 47380 62328 37011
  8890 82236 80035 76231 90746  2305 33836 75675 25701 91413 84878 40983
 92023 70812 25701 52378 81016 15444 71409 69100 56902 58046 73787  3325
 28283 10365 18109 47234 11103 80501 41434 55840 47819 23811 67269 52249
 33836 47236 18835  5387 52249 30401 79063 47819  1035 62319 92273 84521
 68463 33260  2977 53231 79063 80750 91115 81296 10884   992 11633 71708
 10785 10884 33449 55840 10785 54985 34630 33260 39306 36917 48396 48547
 81137     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0], shape=(150,), dtype=int64) tf.Tensor(0, shape=(), dtype=int64)
This was an

In [14]:
#now tokenize/encode/pad all training
# and testing data
encoded_train = imdb_train.map(encode_tf_fn,
                               num_parallel_calls=tf.data.experimental.AUTOTUNE)
encoded_test = imdb_test.map(encode_tf_fn,
                             num_parallel_calls=tf.data.experimental.AUTOTUNE)

# Preparing the model

In [15]:
# Length of the vocabulary in chars
vocab_size = imdb_encoder.vocab_size # len(chars)

# The embedding dimension
embedding_dim = 64

# Number of RNN units
rnn_units = 64

#batch size
BATCH_SIZE=100

In [16]:
def build_model_lstm(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, mask_zero=True,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units),
    tf.keras.layers.Dense(1, activation='sigmoid')
  ])
  return model


In [17]:
model = build_model_lstm(vocab_size = vocab_size,
                         embedding_dim=embedding_dim,
                         rnn_units=rnn_units,
                         batch_size=BATCH_SIZE)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (100, None, 64)           6011584   
_________________________________________________________________
lstm (LSTM)                  (100, 64)                 33024     
_________________________________________________________________
dense (Dense)                (100, 1)                  65        
Total params: 6,044,673
Trainable params: 6,044,673
Non-trainable params: 0
_________________________________________________________________


In [18]:
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy', 'Precision', 'Recall'])

# Model Training and Evaluation

In [19]:
# Prefetch for performance
encoded_train_batched = encoded_train.batch(BATCH_SIZE).prefetch(100)

In [20]:
model.fit(encoded_train_batched, epochs=10)

Epoch 1/10
250/250 [==============================] - 19s 75ms/step - loss: 0.4200 - accuracy: 0.8013 - precision: 0.7810 - recall: 0.8374
Epoch 2/10
250/250 [==============================] - 21s 83ms/step - loss: 0.1716 - accuracy: 0.9388 - precision: 0.9391 - recall: 0.9386
Epoch 3/10
250/250 [==============================] - 20s 82ms/step - loss: 0.1007 - accuracy: 0.9669 - precision: 0.9691 - recall: 0.9646
Epoch 4/10
250/250 [==============================] - 21s 83ms/step - loss: 0.0783 - accuracy: 0.9737 - precision: 0.9738 - recall: 0.9736
Epoch 5/10
250/250 [==============================] - 21s 82ms/step - loss: 0.0437 - accuracy: 0.9852 - precision: 0.9861 - recall: 0.9843
Epoch 6/10
250/250 [==============================] - 21s 83ms/step - loss: 0.0276 - accuracy: 0.9918 - precision: 0.9926 - recall: 0.9910
Epoch 7/10
250/250 [==============================] - 21s 85ms/step - loss: 0.0382 - accuracy: 0.9877 - precision: 0.9881 - recall: 0.9873
Epoch 8/10
250/250 [=======

In [21]:
model.evaluate(encoded_test.batch(BATCH_SIZE))

250/250 [==============================] - 20s 78ms/step - loss: 0.7636 - accuracy: 0.8349 - precision: 0.8458 - recall: 0.8191


[0.7636314034461975, 0.8349199891090393, 0.845848798751831, 0.8191199898719788]

# BiLSTM Model

In [22]:
# Length of the vocabulary in chars
vocab_size = imdb_encoder.vocab_size # len(chars)

# The embedding dimension
embedding_dim = 128

# Number of RNN units
rnn_units = 64

#batch size
BATCH_SIZE=50

In [23]:
dropout=0.2
def build_model_bilstm(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, mask_zero=True,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.Dropout(dropout),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(rnn_units, return_sequences=True, dropout=0.25)),
    tf.keras.layers.Dropout(dropout),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(rnn_units, dropout=0.25)),
    tf.keras.layers.Dropout(dropout),
    tf.keras.layers.Dense(1, activation='sigmoid')
  ])
  return model

In [24]:
bilstm = build_model_bilstm(vocab_size = vocab_size,
                            embedding_dim=embedding_dim,
                            rnn_units=rnn_units,
                            batch_size=BATCH_SIZE)
bilstm.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (50, None, 128)           12023168  
_________________________________________________________________
dropout (Dropout)            (50, None, 128)           0         
_________________________________________________________________
bidirectional (Bidirectional (50, None, 128)           98816     
_________________________________________________________________
dropout_1 (Dropout)          (50, None, 128)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (50, 128)                 98816     
_________________________________________________________________
dropout_2 (Dropout)          (50, 128)                 0         
_________________________________________________________________
dense_1 (Dense)              (50, 1)                  

In [25]:
bilstm.compile(loss='binary_crossentropy', 
               optimizer='adam', 
               metrics=['accuracy', 'Precision', 'Recall'])

In [26]:
encoded_train_batched = encoded_train.batch(BATCH_SIZE).prefetch(100)

In [27]:
bilstm.fit(encoded_train_batched, epochs=5)

Epoch 1/5
500/500 [==============================] - 64s 128ms/step - loss: 0.3738 - accuracy: 0.8256 - precision: 0.8204 - recall: 0.8337
Epoch 2/5
500/500 [==============================] - 66s 132ms/step - loss: 0.1467 - accuracy: 0.9462 - precision: 0.9467 - recall: 0.9456
Epoch 3/5
500/500 [==============================] - 65s 131ms/step - loss: 0.0745 - accuracy: 0.9753 - precision: 0.9757 - recall: 0.9749
Epoch 4/5
500/500 [==============================] - 66s 132ms/step - loss: 0.0515 - accuracy: 0.9823 - precision: 0.9829 - recall: 0.9817
Epoch 5/5
500/500 [==============================] - 66s 132ms/step - loss: 0.0270 - accuracy: 0.9902 - precision: 0.9905 - recall: 0.9899


In [28]:
bilstm.evaluate(encoded_test.batch(BATCH_SIZE))

500/500 [==============================] - 16s 32ms/step - loss: 0.6841 - accuracy: 0.8360 - precision: 0.8250 - recall: 0.8529


[0.6841403245925903,
 0.8360000252723694,
 0.8250271081924438,
 0.8528800010681152]

In [29]:
# Release GPU memory
from numba import cuda

cuda.select_device(0)
cuda.close()